<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_MultiTimeFrame_RSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-05-03 22:14:31


In [6]:
# install libraries
!pip install yfinance
!pip install pandas-ta

In [7]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd

#mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')
  return mypps_df

mypps_df =  get_mypps_df()
mypfs_df =  get_mypfs_df()

def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

def get_stock_rsi(symbols, period='5y'):
    result = []
    for symbol in symbols:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol + ".NS", period=period)

        close_price = round(stock_data.tail(1)['Close'].values[0],2)
        #print(close_price)
        all_time_high = round(max(stock_data['Close']),2)
        #print(all_time_high)
        ath_perc = round((((all_time_high - close_price)*100)/close_price),0)
        #print(ath_perc)

        # Calculate daily RSI
        stock_data.ta.rsi(close='Close', length=14, append=True)

        # Calculate weekly RSI
        weekly_data = stock_data.resample('W').last()
        weekly_data.ta.rsi(close='Close', length=14, append=True)

        # Calculate monthly RSI
        monthly_data = stock_data.resample('M').last()
        monthly_data.ta.rsi(close='Close', length=14, append=True)

        # Extract the last value of RSI for each timeframe
        daily_rsi = round(stock_data['RSI_14'].iloc[-1], 0)
        weekly_rsi = round(weekly_data['RSI_14'].iloc[-1], 0)
        monthly_rsi = round(monthly_data['RSI_14'].iloc[-1], 0)

        # Append the results to the list
        result.append({
            'Stock_Symbol': symbol,
            'Daily_RSI': daily_rsi,
            'Weekly_RSI': weekly_rsi,
            'Monthly_RSI': monthly_rsi,
            'ATH%' : ath_perc
        })

    # Create a DataFrame with the results
    result_df = pd.DataFrame(result)
    return result_df

if __name__ == "__main__":
    # Example usage
    mypps_lst = get_stock_ids(mypps_df)
    symbols = mypps_lst # Example NSE stock symbols
    stock_rsi = get_stock_rsi(symbols)
stock_rsi

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Stock_Symbol,Daily_RSI,Weekly_RSI,Monthly_RSI,ATH%
0,3MINDIA,45.0,45.0,54.0,26.0
1,5PAISA,45.0,50.0,62.0,40.0
2,AARTIIND,59.0,66.0,60.0,56.0
3,AAVAS,58.0,56.0,46.0,101.0
4,ABBOTINDIA,45.0,52.0,63.0,13.0
...,...,...,...,...,...
225,WHIRLPOOL,58.0,58.0,47.0,83.0
226,WIPRO,38.0,47.0,53.0,58.0
227,WSTCSTPAPR,71.0,55.0,59.0,15.0
228,YESBANK,50.0,55.0,59.0,605.0


In [8]:
tmp_df = stock_rsi[(stock_rsi['Daily_RSI'] > 60) & (stock_rsi['Daily_RSI'] < 70)]
tmp_df = tmp_df[(stock_rsi['Weekly_RSI'] > 55) & (tmp_df['Weekly_RSI'] < 65)]
tmp_df = tmp_df[(stock_rsi['Monthly_RSI'] > 50) & (tmp_df['Monthly_RSI'] < 60)]
tmp_df['InPortfolio'] = tmp_df['Stock_Symbol'].isin(mypfs_df['Symbol']).astype(int)
tmp_df.sort_values(by = 'ATH%', ascending=False)

<ipython-input-8-40b36d755ed8>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_df = tmp_df[(stock_rsi['Weekly_RSI'] > 55) & (tmp_df['Weekly_RSI'] < 65)]
<ipython-input-8-40b36d755ed8>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_df = tmp_df[(stock_rsi['Monthly_RSI'] > 50) & (tmp_df['Monthly_RSI'] < 60)]


,Stock_Symbol,Daily_RSI,Weekly_RSI,Monthly_RSI,ATH%,InPortfolio
103,IEX,64.0,59.0,54.0,87.0,1
59,DCBBANK,65.0,58.0,59.0,74.0,0
37,BIOCON,65.0,62.0,55.0,58.0,1
57,CUB,67.0,63.0,55.0,52.0,0
62,DIVISLAB,65.0,59.0,56.0,36.0,0
149,NEOGEN,68.0,60.0,55.0,24.0,1
